In [1]:
import os
import random

import torch
from PIL import Image
from diffusers import KandinskyV22PriorEmb2EmbPipeline
from diffusers.utils import make_image_grid

from models.combined_pipeline_prior import CombinedPipelineV2


In [2]:
import tomesd

decoder = CombinedPipelineV2.from_pretrained("../models/weights/img2img-painting", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
decoder.enable_model_cpu_offload()
decoder.enable_attention_slicing()
decoder.enable_xformers_memory_efficient_attention()

decoder = tomesd.apply_patch(decoder, ratio=0.5)

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [3]:
prior = KandinskyV22PriorEmb2EmbPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior',
                                                         image_encoder=decoder.image_encoder,
                                                         image_processor=decoder.image_processor,
                                                         torch_dtype=torch.float16)
prior.enable_model_cpu_offload()
prior.enable_attention_slicing()
prior.cuda()

text_encoder\model.safetensors not found


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
negative_prior_prompt ='lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature'

clothes = ['shirt', 'pants', 'shorts', 'shoes', 't-shirt',
            'sweatshirt', "cowboy boot", "cowboy hat",
            'crash helmet', 'cardigan', 'blouse', 'jacket', 'jeans',]
torch.manual_seed(42)

In [5]:
torch.no_grad()
def generate_with_prior(source_image, target_image):
    clip_img_emb = prior.interpolate(images_and_prompts=[source_image, target_image],
                                     weights=[1, .8],
                                     num_images_per_prompt=1).image_embeds
    source_image_embeds = prior(prompt='a human being wearing some clothes in front of a white background',
                    image=clip_img_emb,
                    strength=0.85,
                    num_inference_steps=25,
                    num_images_per_prompt=1,)
    source_negative_embeds = prior(prompt=negative_prior_prompt,
                         image=clip_img_emb,
                         strength=1,
                         num_inference_steps=25,
                         num_images_per_prompt=1)
    target_image_embeds = prior(prompt='a clothing worn in front of a white background',
                image=clip_img_emb,
                strength=0.85,
                num_inference_steps=25,
                num_images_per_prompt=1,)
    target_negative_embeds = prior(prompt=negative_prior_prompt,
                         image=clip_img_emb,
                         strength=1,
                         num_inference_steps=25,
                         num_images_per_prompt=1)

    images = decoder(source_image,
                     target_image,
                     source_image_embeds=source_image_embeds.image_embeds,
                     source_negative_image_embeds=source_negative_embeds.image_embeds,
                     target_image_embeds=target_image_embeds.image_embeds,
                     target_negative_image_embeds=target_negative_embeds.image_embeds,
                     source_prompt=clothes,
                     source_negative_prompt=['head, neck', 'hair', 'face',],
                     target_prompt=clothes,
                     target_negative_prompt=['head, neck', 'hair', 'face', 'hand',],
                     num_inference_steps=50, strength=0.5,height=512, width=512)
    return images


In [6]:
input_folder = '../cache/in/'
output_folder = '../cache/out/'

In [7]:
paths = os.listdir(input_folder)
random.shuffle(paths)

images = [Image.open(input_folder + path) for path in paths]
generated_images, couple = [], []

for i in range(len(images)):
    for j in range(i + 1, len(images)):
        images[i], images[j] = images[i].resize((512, 512)), images[j].resize((512, 512))
        generated_images.extend([
            make_image_grid([*generate_with_prior(images[i], images[j]),
                             images[i], images[j]], 3, 2),
            make_image_grid([*generate_with_prior(images[j], images[i]),
                             images[j], images[i]], 3, 2)
        ])

  0%|          | 0/21 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 6.00 GiB total capacity; 4.91 GiB already allocated; 0 bytes free; 5.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF